In [3]:
import pandas as pd 
import numpy as np

In [5]:
data = pd.read_csv('HURdat_ExtremeWeatherEvents.csv')
data = data[list(data.columns)[1:]]
data.head(10)

,ID,Name,Status,Latitude,Longitude,Maximum.Wind,date_time,diff,rapid_int,i,...,persistence,product,Initial.Max,speed,speed_z,speed_m,Jday,Maximum.Wind_p,Latitude_p,Longitude_p
0,AL011980,UNNAMED,TD,30.5,-76.5,20,7/17/1980 0:00,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,55,NaN,NaN,NaN
1,AL011980,UNNAMED,TD,30.2,-77.2,25,7/17/1980 6:00,NaN,NaN,2,...,NaN,NaN,20.0,NaN,NaN,NaN,55,20.0,30.5,-76.5
2,AL011980,UNNAMED,TD,30.0,-78.0,25,7/17/1980 12:00,NaN,NaN,3,...,NaN,NaN,25.0,NaN,NaN,NaN,55,25.0,30.2,-77.2
3,AL011980,UNNAMED,TD,29.9,-79.3,25,7/17/1980 18:00,NaN,NaN,4,...,5.0,125.0,25.0,12.869511,0.035365,9.266244,55,25.0,30.0,-78.0
4,AL011980,UNNAMED,TD,29.8,-80.2,25,7/18/1980 0:00,5.0,0.0,5,...,0.0,0.0,25.0,17.071172,0.035158,9.266244,54,25.0,29.9,-79.3
5,AL011980,UNNAMED,TD,29.5,-81.1,25,7/18/1980 6:00,0.0,0.0,6,...,0.0,0.0,25.0,17.768950,0.035018,9.266244,54,25.0,29.8,-80.2
6,AL011980,UNNAMED,TD,29.5,-82.0,20,7/18/1980 12:00,-5.0,0.0,7,...,0.0,0.0,25.0,14.954518,0.034947,9.266244,54,25.0,29.5,-81.1
7,AL011980,UNNAMED,TD,29.5,-84.7,20,7/19/1980 6:00,-5.0,0.0,10,...,0.0,0.0,25.0,16.164193,0.034660,9.266244,53,20.0,29.4,-84.0
8,AL011980,UNNAMED,TD,29.6,-85.8,25,7/19/1980 12:00,5.0,0.0,11,...,0.0,0.0,25.0,13.748279,0.034587,9.266244,53,20.0,29.5,-84.7
9,AL011980,UNNAMED,TD,29.7,-86.4,25,7/19/1980 18:00,5.0,0.0,12,...,5.0,125.0,25.0,14.634529,0.034587,9.266244,53,25.0,29.6,-85.8


In [70]:
# Pandas(Index=153, ID='AL011993', Name='UNNAMED', Status='TD', 
# Latitude=26.7, Longitude=-74.0, _6=30, date_time='6/2/1993 0:00', 
# diff=5.0, rapid_int=0.0, i=7, n=11, persistence=5.0, product=150.0, 
# _14=30.0, speed=30.632630100000004, speed_z=0.030517206, speed_m=9.266243887, 
# Jday=101, _19=30.0, Latitude_p=26.1, Longitude_p=-75.8)

# max distinct 8372 states
triples = set()
previous_status = None
previous_name = None
states = set()
for state in data.itertuples():
    # Skip if we don't have the data
    if str(state.speed) == 'nan' or str(state.product) == 'nan':
        continue
    # Get the name
    name = state.ID
    # Features
    latitude = state.Latitude
    longitude = state.Longitude
    speed = state.speed
    speed_z = state.speed_z
    speed_m = state.speed_m
    product = state.product
    # Create a status string
    status = "state_{lat}_{long}_{speed}_{speed_z}_{speed_m}_{product}".format(
        lat=round(latitude / 10.0),
        long=round(longitude / 10.0),
        speed=round(speed),
        speed_z=round(speed_z),
        speed_m=round(speed_m),
        product=round(product)
    )
    # Add extra triple to connect the statuses
    lat = 'lat_{}'.format(round(latitude))
    lat_zone = 'lat_{}'.format(round(latitude / 10))
    triples.add('{s},{p},{o}'.format(s=status, p='has_lat', o=lat))
    triples.add('{s},{p},{o}'.format(s=lat, p='in_lat_zone', o=lat_zone))
    long = 'long_{}'.format(round(longitude))
    long_zone = 'long_{}'.format(round(longitude / 10))
    triples.add('{s},{p},{o}'.format(s=status, p='has_long', o=long))
    triples.add('{s},{p},{o}'.format(s=long, p='in_long_zone', o=long_zone))
    # Add the triples to link the states and prepare for next
    if previous_name != name:
        previous_name = None
    if previous_status != None and previous_name == name:
        p = 'escalate_to' if str(state.rapid_int)=='1.0' else 'transit_to'
        triples.add('{s},{p},{o}'.format(s=previous_status, p=p, o=status))
    previous_status = status
    previous_name = name
    states.add(state)
print (len(states))
print (list(states)[1])
print (len(triples))

7515
Pandas(Index=5943, ID='AL121981', Name='            UNNAMED', Status=' TD', Latitude=26.0, Longitude=-99.0, _6=-99, date_time='8/29/1981 12:00', diff=-124.0, rapid_int=0.0, i=12, n=12, persistence=0.0, product=0.0, _14=30.0, speed=12.21457387, speed_z=0.030049607000000002, speed_m=9.266243887, Jday=13, _19=25.0, Latitude_p=25.2, Longitude_p=-98.8)
20209


In [72]:
df = pd.DataFrame([r.split(',') for r in triples], columns=['s', 'p', 'o'])
df.to_csv('triples.csv', index=False)
df

,s,p,o
0,state_2_-5_22_0_9_175,has_lat,lat_22
1,state_2_-5_31_0_9_0,has_long,long_-54
2,state_2_-5_29_0_9_250,transit_to,state_2_-5_28_0_9_0
3,state_1_-8_21_0_9_500,transit_to,state_1_-8_24_0_9_0
4,state_1_-4_30_0_9_0,transit_to,state_1_-4_28_0_9_0
...,...,...,...
20204,state_2_-4_19_0_9_200,has_long,long_-41
20205,state_4_-4_39_0_9_350,has_lat,lat_36
20206,state_3_-5_32_0_9_0,has_lat,lat_25
20207,state_1_-6_36_0_9_450,transit_to,state_1_-6_36_0_9_225
